In [11]:
import os
import pickle
import numpy as np
from statsmodels.tsa.api import VAR
from sklearn.metrics import mean_squared_error as mse

In [12]:
### Load data. The data is part of HCP dataset, contains "match" block in relational task run 1
### Dimension of the data is 1017 * 23 * 360 (#trials * MRI frames * cortex parcels)

with open('AR_data.pkl', 'rb') as fh:
    relational_HCP = pickle.load(fh)[0]

np.shape(relational_HCP)

(1017, 23, 360)

In [13]:
### Functions to run and test autoregression model

def run_ar(data, lag):
  '''
  Running autoregression model on the data.

  Input:
  data: # of trials * frames * parcels
  lag: int

  Return:
  coef: # of trials * lag * parcels * parcels
  sigma_u: # of trials * parcels * parcels

  '''

  coef = []
  sigma_u = []

  num = len(data)
  for ii in range(num):
    if ii%100 == 0:
      print(f"{ii} / {num}")

    # if np.shape(data[ii]) != shape:
    #   continue

    curr_model = VAR(data[ii])
    curr_result = curr_model.fit(lag)

    coef.append(curr_result.coefs)
    sigma_u.append(curr_result.sigma_u)
  
  return coef, sigma_u

def test_ar(data, coef, sigma_u, lag, prop):
  '''
  Randomly choose a set of test data for testing model coef.
  Baseline preformance is persistance model (x_t = x_{t-1}).
  Print the number of datapoints where model is better, and the average baseline and model mse.

  Input:
  data: # of trials * frames * parcels
  coef: # of trials * lag * parcels * parcels, output from run_ar
  sigma_u: # of trials * parcels * parcels, output from run_ar
  lag: int
  prop: proportion of data to be used as test data

  Output:
  baseline_mse: mse of baseline performance
  all_mse: mse of model on test data

  '''

  num = len(data)
  nframe = len(data[0])
  all_mse = []
  baseline_mse = []

  # idx for test and train data, split test data
  idx = np.random.randint(low = 0, high = int(1/prop), size = num)
  idx_test = np.where(idx==0)[0]
  idx_train = np.where(idx!=0)[0]
  data_test = [data[x] for x in idx_test]

  # params from training set
  coef_train = [coef[x] for x in idx_train]
  coef_train = np.average(coef_train, axis = 0)
  sigma_u_train = [sigma_u[x] for x in idx_train]
  sigma_u_train = np.average(sigma_u_train, axis = 0)

  # baseline performance
  for jj in range(len(idx_test)):
    true = np.transpose(data_test[jj][lag:])
    baseline_pred = np.transpose(data_test[jj][lag-1:-1])
    baseline_mse.append(mse(true, baseline_pred))

  # use coef of training set to test
  if lag == 1:
    for ii in range(len(idx_test)):

      pred = np.dot(coef_train[0], np.transpose(data_test[ii][:-1]))
      noise = np.random.multivariate_normal(mean = np.zeros(360), cov = sigma_u_train, size = nframe-lag)
      pred = pred + noise.transpose()
      all_mse.append(mse(data_test[ii][1:], pred.transpose()))

  elif lag != 1:
    for ii in range(len(idx_test)):
      pred = []

      for jj in range(nframe-lag):
        temp = []

        for kk in range(lag):
          temp.append(np.dot(coef_train[kk], np.transpose(data_test[ii][kk+jj])))
        
        pred.append(np.sum(temp, axis = 0))
      
      noise = np.random.multivariate_normal(mean = np.zeros(360), cov = sigma_u_train, size = nframe-lag)
      pred = pred + noise
      all_mse.append(mse(data_test[ii][lag:], pred))

  compare = np.less(all_mse, baseline_mse)
  print(f"Num of model mse less than baseline mse: {sum(compare)}/{len(idx_test)}")

  baseline_mse = np.average(baseline_mse)
  all_mse = np.average(all_mse)

  print(f"Average baseline mse: {baseline_mse}\nAverage model mse: {all_mse}")

  # return baseline_mse, all_mse

In [14]:
### Run the model for many lag value on the data
np.random.seed(2020)
for lag in range(1,5):
  print(f"Current lag num: {lag}\nRunning AR:")
  curr_coef, curr_sigma_u = run_ar(relational_HCP, lag = lag)
  print("Testing AR:")
  test_ar(data = relational_HCP, coef = curr_coef, sigma_u = curr_sigma_u, lag=lag, prop=0.2)
  print('\n')

Current lag num: 1
Running AR:
0 / 1017
100 / 1017
200 / 1017
300 / 1017
400 / 1017
500 / 1017
600 / 1017
700 / 1017
800 / 1017
900 / 1017
1000 / 1017
Testing AR:
Num of model mse less than baseline mse: 192/227
Average baseline mse: 5088.078272915777
Average model mse: 4151.6896841533835


Current lag num: 2
Running AR:
0 / 1017
100 / 1017
200 / 1017
300 / 1017
400 / 1017
500 / 1017
600 / 1017
700 / 1017
800 / 1017
900 / 1017
1000 / 1017
Testing AR:
Num of model mse less than baseline mse: 165/221
Average baseline mse: 5075.453348951435
Average model mse: 4590.44222082113


Current lag num: 3
Running AR:
0 / 1017
100 / 1017
200 / 1017
300 / 1017
400 / 1017
500 / 1017
600 / 1017
700 / 1017
800 / 1017
900 / 1017
1000 / 1017
Testing AR:
Num of model mse less than baseline mse: 168/208
Average baseline mse: 5582.409717501336
Average model mse: 4688.052827868265


Current lag num: 4
Running AR:
0 / 1017
100 / 1017
200 / 1017
300 / 1017
400 / 1017
500 / 1017
600 / 1017
700 / 1017
800 / 1017